In [53]:
import requests
import json
import pandas as pd

### Get a user's playlists

In [54]:
def get_plst_ID(user_id, token):
    tidy = lambda s: s[17:]
    base_url = "https://api.spotify.com/v1/users/"
    query = f'{base_url}{user_id}/playlists'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['uri'])for x in json_response['items']]

### Get a Playlist's Items

In [55]:
def get_ID(user_id, token, playlist_id, market = 'US'):
    tidy = lambda s: s[14:]
    base_url = "https://api.spotify.com/v1/playlists/"
    query = f'{base_url}{playlist_id}/tracks?market={market}'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['track']['uri'])for x in json_response['items']], [json_response['items'][i]['track']['duration_ms'] for i in range(len(json_response['items']))]

### Get information about a song

In [56]:
def get_song(user_id, token, song_id, market = 'US'):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    response = requests.get(query, 
                   headers={"Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    album_id = json_response['album']['id']
    artist_id = json_response['artists'][0]['id']
    
    ## get genre of the album
    base_url = "https://api.spotify.com/v1/albums/"
    query = f'{base_url}{album_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    genre = json_response['genres']
    
    
    df_response['artist_id'] = artist_id if len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if len(album_id) >0 else None
    df_response['genre'] = genre if len(genre) >0 else None
    
    return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')

### Get Recommendations from Spotify API

In [75]:
def recommend(song, limit, time_mt, percent, target_duration_ms, **kwargs):
    seed_tracks = song.index[0]  #******** WHY ARE ALL SONG.INDEX[] using position [0]??????  Is thre only one row of data?

    min_duration_ms = max(target_duration_ms - time_mt * 1000, 0)
    max_duration_ms = target_duration_ms + time_mt * 1000 
    
    target_energy = song['energy'][0]  ## find the most similar one to the target
    min_energy = max(target_energy * (1 - percent), 0)
    max_energy = min(target_energy * (1 + percent),1)
    
    target_instrumentalness = song['instrumentalness'][0]  ## similarly to energy
    min_instrumentalness = max(target_instrumentalness * (1 - percent*3), 0)
 #   max_instrumentalness = min(target_instrumentalness * (1 + percent*2),1)
    
    target_tempo = song['tempo'][0]  ## similarly to energy
    min_tempo = max(target_tempo * (1 - percent), 0)
    max_tempo = target_tempo * (1 + percent)
    target_key = song['key'][0]
    target_danceability=song['danceability'][0]
    target_mode=song['mode'][0]
    
    endpoint_url = "https://api.spotify.com/v1/recommendations?"
    query = f'{endpoint_url}limit={limit}&seed_tracks={seed_tracks}'
    query += f'&target_duration_ms={target_duration_ms}&min_duration_ms={min_duration_ms}&max_duration_ms={max_duration_ms}'
    query += f'&target_energy={target_energy}&min_energy={min_energy}&max_energy={max_energy}'
    query += f'&target_instrumentalness={target_instrumentalness}&min_instrumentalness={min_instrumentalness}'
    query += f'&target_tempo={target_tempo}&min_tempo={min_tempo}&max_tempo={max_tempo}'
    query += f'&target_key={target_key}'
    query += f'&target_danceability={target_danceability}'
    query += f'&target_mode={target_mode}'
    
    if song['genre'][0] is not None:
        seed_genres = song['genre'][0]
        query += f'&seed_genres={seed_genres}'
#        query += f'&seed_genres={"dance"}'
    if song['artist_id'][0] is not None:
        seed_artist = song['artist_id'][0]
        query += f'&seed_artist={seed_artist}'
    
    if len(kwargs.keys()) > 0:
        query += '&'
        lst = [str(x[0])+'='+str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all limits from kwargs
    uris = [] 
    #print(query)
    
    response = requests.get(query, 
               headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    json_response = response.json()
    print('Recommended Songs:')  #**uncommented
    uris = []
    for i,j in enumerate(json_response['tracks']):
        uris.append(j['uri'])
        print(f"{i+1}) \"{j['name']}\" by {j['artists'][0]['name']}") #**uncommented
    return uris

### Filter recommended songs

In [76]:
#uris

### Create a playlist

In [77]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

### generate a similar playlist

In [78]:
def get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, time_percent, percent, public=False, **kwargs):
    print(playlist_id)
    market = kwargs.get("market", 'US')
    
    def _get_one_recommendation(idx, target_duration_ms, **kwargs):
        df_response = get_song(user_id, token, idx, market)
        r = recommend(df_response, limit, time_percent, percent, target_duration_ms, market=market, **kwargs)

        print(f"For the {j+1}-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.")
        new_time_percent = time_percent
        new_percent = percent
        i = 0
        while len(r) == 0 and i < 10:
            i += 1
            new_time_percent += 10
            new_percent *= 2
            r = recommend(df_response, limit, new_time_percent, new_percent, target_duration_ms, market=market, **kwargs)
        if i >= 10:
            ## right now I keep the original one, we can improve it later
            r = ['spotify:track:' + df_response.index[0]]
        return r
        
    uris = []
    duration = 0
    ids, durations = get_ID(user_id, token, playlist_id, market)
    diff = 0
    for j in range(len(ids)):
        r = _get_one_recommendation(ids[j], target_duration_ms=durations[j] - diff, **kwargs)   
        print(r)
        _r = filter_energy(r) ## add limitations to filter the recommendations and keep the energy trend
        uris.append(_r)
        dur = get_song(user_id, token, r[0][14:], market)['duration_ms'][0]
        duration += dur
        diff = duration - sum(durations[:j+1])
    
    create_plst(user_id, token, [x for x in uris if x is not None], name, description, public)

## Test

Note: token should have scope at least "playlist-modify-private", you can find get one at the botton GET TOKEN from [here](https://developer.spotify.com/console/get-recommendations/?limit=10&market=ES&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical%2Ccountry&seed_tracks=0c6xIDDpzE81m2q797ordA&min_acousticness=&max_acousticness=&target_acousticness=&min_danceability=&max_danceability=&target_danceability=&min_duration_ms=&max_duration_ms=&target_duration_ms=&min_energy=&max_energy=&target_energy=&min_instrumentalness=&max_instrumentalness=&target_instrumentalness=&min_key=&max_key=&target_key=&min_liveness=&max_liveness=&target_liveness=&min_loudness=&max_loudness=&target_loudness=&min_mode=&max_mode=&target_mode=&min_popularity=&max_popularity=&target_popularity=&min_speechiness=&max_speechiness=&target_speechiness=&min_tempo=&max_tempo=&target_tempo=&min_time_signature=&max_time_signature=&target_time_signature=&min_valence=&max_valence=&target_valence=).

User id can be obtained from [your Spotify profile](https://www.spotify.com/us/account/overview/?utm_source=spotify&utm_medium=menu&utm_campaign=your_account).

In [86]:
# settings
token = "BQCVP-8dyStT2HPX1qcOPkjOn3DB5eWah9rL6Kuiz4X9LcSQQxgQ_BSvBIqLNpcvVn9I6denRfIl82pMruU1VGkZjEmAGOqwIk4ndzramSOFg87cMMjYD7gu0Yl7sXpLRzta1ip-4XoK0s0FrrtkRpk7ZArPGnebyA-cuQ1der6YScxtjU1ggTwZEonyzPbn5j1m4_gDLjip-C3hjHAwAwmbsf3SDn2YhASc0s6hyzWBqsT5YbRm0ACqsTJ_mAXjnzSEBGnbtr-zxv85j1REUEeD"
user_id = "2272ss2bp3tznxg6k3v63kqxq"

name = 'Test YS PL'   ## name for the new playlist
description = 'recommendations based on Seed PL'   ## description for the new playlist
limit = 5  ## for each song in current playlist, 5 recommendations are generated
time_percent = 20  ## the time interval width is 30s
percent = 0.1  ## the energy is current song's energy * (0.9, 1.1)

plst_id = get_plst_ID(user_id, token)  ## get id for each playlist from a user
playlist_id = plst_id[1]    ## set the 1st playlist to be the target playlist

In [87]:
# run it
get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, time_percent, percent)

5RZl9RxEUwA2KT0A9838ls
Recommended Songs:
1) "Let the Music Play" by Santi Garcia
2) "Crush - Extended Mix" by Deepjack
For the 1-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.
Recommended Songs:
1) "Sensual Strawberry Soda - Tommy Largo Remix" by Peckos
2) "Oregon - Shingo Nakamura Mix" by Shingo Nakamura
3) "Make A Livin' - 2020 Relick" by Jay Vegas
4) "I Won't Let You Go - Club Mix" by Anthony Romeno
5) "Chrystallum - Mixed" by Shingo Nakamura
For the 2-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.
Recommended Songs:
For the 3-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
Recommended Songs:
1) "Up & Down" by Purple

Recommended Songs:
1) "About Us" by Le Youth
2) "Hundred Fifty Up" by Satin Jackets
For the 30-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.
Recommended Songs:
1) "I Feel Love" by Kevin McKay
For the 31-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.
Recommended Songs:
1) "Retroverb - Original Mix" by Superlover
2) "P.O.W.E.R - Original Mix" by Nytron
3) "So Good" by Sandy Rivera
4) "Make Your Move - Original Mix" by Croatia Squad
5) "Black Betty [Remix 2016]" by Jl & Afterman
For the 32-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.
Playlist Test C2 AB is successfully created!
Playlist Test C2 AB is successfully filled with recommendations!
Your playlist is ready at https://open.spotify.com/playlist/13bn7nqXpHn6HxK4dVmxA5


#### Check your new playlist in your Spotify!